In [ ]:
# TO DO:
# General: add second leg, etc.
# Rel Specific: recheck simplifications, benchmark

In [ ]:
# DERIVE THE EOMs SYMBOLICALLY -------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np
import multiprocessing as mp #multiprocessing, just for the simplification steps
import time # for benchmarking different models

sym.init_printing()
from IPython.display import display #for pretty printing

# define non--program-critical functions
def log_time(t_start=None, msg=""):
    t_end = time.perf_counter()
    if t_start == None: print("Timer initialized")
    else: print(msg + f": {t_end - t_start:.6f} sec")
    return t_end

def mp_trigsimp(m):
    mp_data = [(m[r,c]) for r in range(m.rows) for c in range(m.cols)]
    if __name__ == '__main__':
        p = mp.Pool()
        results = p.map(sym.trigsimp, mp_data)
        p.close()
        p.join()
    return sym.Matrix(results).reshape(m.rows,m.cols)

# ------------------------------------------------------------------------------------------------------------------
# create symbolic variables

# system parameters 
g = sym.symbols('g')
m1, l1, In1, d1 = sym.symbols(['m_{1}','l_{1}','I_{1}','d_{1}'])
m2, l2, In2, d2 = sym.symbols(['m_{2}','l_{2}','I_{2}','d_{2}'])
m3, l3, In3, d3 = sym.symbols(['m_{3}','l_{3}','I_{3}','d_{3}'])
m4, l4, In4, d4 = sym.symbols(['m_{4}','l_{4}','I_{4}','d_{4}'])



# generalized coordinates
x  ,  dx,  ddx = sym.symbols([          'x',          '\dot{x}',          '\ddot{x}'])
y  ,  dy,  ddy = sym.symbols([          'y',          '\dot{y}',          '\ddot{y}'])
th1,dth1,ddth1 = sym.symbols(['\\theta_{1}','\dot{\\theta}_{1}','\ddot{\\theta}_{1}'])

sides = ['L','R']

th2 = [sym.symbols('\\theta_{2%s}'%s) for s in sides]
th3 = [sym.symbols('\\theta_{3%s}'%s) for s in sides]
th4 = [sym.symbols('\\theta_{4%s}'%s) for s in sides]

dth2 = [sym.symbols('\dot{\\theta}_{2%s}'%s) for s in sides]
dth3 = [sym.symbols('\dot{\\theta}_{3%s}'%s) for s in sides]
dth4 = [sym.symbols('\dot{\\theta}_{4%s}'%s) for s in sides]

ddth2 = [sym.symbols('\ddot{\\theta}_{2%s}'%s) for s in sides]
ddth3 = [sym.symbols('\ddot{\\theta}_{3%s}'%s) for s in sides]
ddth4 = [sym.symbols('\ddot{\\theta}_{4%s}'%s) for s in sides]

q   = sym.Matrix(  [x] +   [y] +   [th1] +   th2 +   th3 +   th4)
dq  = sym.Matrix( [dx] +  [dy] +  [dth1] +  dth2 +  dth3 +  dth4)
ddq = sym.Matrix([ddx] + [ddy] + [ddth1] + ddth2 + ddth3 + ddth4)


# ------------------------------------------------------------------------------------------------------------------
# functions, # don't ask, they work for the purposes of this program
def WeirdRot(v, th):
    # weird rotation function for working with [x; y; th] vectors
    # usage: gives R_1^0 transformation, and adds the angle of rotation to result
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [       0,        0, 1]])
    S = sym.Matrix([[0],[0],[th]])
    return R*v + S

def WeirdAdd(v1, v2):
    # returns the sum of two vectors of the form [x; y; th]
    # both in the same ref. frame, preserving only the first angle
    v3 = sym.Matrix([[1],[1],[0]])
    return v1 + sym.matrix_multiply_elementwise(v3, v2)

def WeirdTrans(v1, ang1, off1, ang2=0, off2=sym.Matrix([[0],[0],[0]])):
    # ^ & ^ just like me
    # transforms [x,y,th] vector to inertial frame using position offsets and
    # rotation angles as specified
    rn_m = WeirdAdd(WeirdRot(v1, ang1),off1)
    rn_0 = WeirdAdd(WeirdRot(rn_m, ang2),off2)
    return rn_0


# ------------------------------------------------------------------------------------------------------------------
# mass positions, foot positions
r1_1 = sym.Matrix([[0]    ,[ l1*d1],[0]]) #sanity check
r2_2 = sym.Matrix([[0]    ,[-l2*d2],[0]])
r3_3 = sym.Matrix([[0]    ,[-l3*d3],[0]])
r4_4 = sym.Matrix([[l4*d4],[0]     ,[0]]) # could define in x, but for consistency -y is fine

rhip_2   = sym.Matrix([[0]    ,[0]     ,[0]]) #sanity check
rknee_3  = sym.Matrix([[0]    ,[-l2],   [0]])
rankle_4 = sym.Matrix([[0]    ,[-l3],   [0]])
rfoot_4  = sym.Matrix([[l4]   ,[0]     ,[0]])

process_timer = log_time()

r1_0 = WeirdTrans(r1_1, th1, sym.Matrix([[x],[y],[0]]))
r2_0 = []
r3_0 = []
r4_0 = []
rhip_0   = []
rknee_0  = []
rankle_0 = []
rfoot_0  = []

for i in range(len(sides)):
    r2_0.append(WeirdTrans(r2_2, th2[i]+th1 , sym.Matrix([[x],[y],[0]])))
    r3_0.append(WeirdTrans(r3_3, th3[i]     , r2_2*(1-d2)/d2           , th1+th2[i]       , r2_0[i]))
    r4_0.append(WeirdTrans(r4_4, th4[i]     , r3_3*(1-d3)/d3           , th1+th2[i]+th3[i], r3_0[i]))
    
    rhip_0.append(  WeirdTrans(rhip_2  , th2[i]+th1 , sym.Matrix([[x],[y],[0]]))) #sanity check
    rknee_0.append( WeirdTrans(rknee_3 , th3[i]     , r2_2*(1-d2)/d2           , th1+th2[i]       , r2_0[i])) #sanity check
    rankle_0.append(WeirdTrans(rankle_4, th4[i]     , r3_3*(1-d3)/d3           , th1+th2[i]+th3[i], r3_0[i]))
    rfoot_0.append( WeirdTrans(rfoot_4 , th4[i]     , r3_3*(1-d3)/d3           , th1+th2[i]+th3[i], r3_0[i]))

    r2_0[i]    = sym.trigsimp(   r2_0[i])
    r3_0[i]    = sym.trigsimp(   r3_0[i])
    r4_0[i]    = sym.trigsimp(   r4_0[i])
    
    rhip_0[i]   = sym.trigsimp(rhip_0[i])
    rknee_0[i]  = sym.trigsimp(rknee_0[i])
    rankle_0[i] = sym.trigsimp(rankle_0[i])
    rfoot_0[i]  = sym.trigsimp(rfoot_0[i])

process_timer = log_time(process_timer, "Simplified 'r's")

# mass velocities, foot velocity
dr1 = r1_0.jacobian(q)*dq
dr2 = []
dr3 = []
dr4 = []
drfoot = []

for i in range(len(sides)):
    dr2.append(      r2_0[i].jacobian(q)*dq)
    dr3.append(      r3_0[i].jacobian(q)*dq)
    dr4.append(      r4_0[i].jacobian(q)*dq)
    drfoot.append(rfoot_0[i].jacobian(q)*dq)

# ------------------------------------------------------------------------------------------------------------------
# kinetic and potential energy
def Ep(m, r):
    return sym.Matrix([m*g*r[1]])

def Ek(m, In, dr):
    InM = sym.Matrix([[m,0,0],[0,m,0],[0,0,In]])
    return 0.5*dr.T*InM*dr

V = Ep(m1,r1_0)
T = [Ek(m1,In1,dr1)]

for i in range(len(sides)):
    V += Ep(m2,r2_0[i]) + Ep(m3,r3_0[i]) + Ep(m4,r4_0[i])
    
    T.append(Ek(m2,In2,dr2[i]))
    T.append(Ek(m3,In3,dr3[i]))
    T.append(Ek(m4,In4,dr4[i]))

T = sym.Matrix(T)
T = mp_trigsimp(T)
T = sum(T[i] for i in range(len(T)))
T = sym.Matrix([T])
process_timer = log_time(process_timer, "Simplified T")



# ------------------------------------------------------------------------------------------------------------------
# forces & torques
tau1 = sym.symbols(['\\tau_{1%s}'%s for s in sides])
tau2 = sym.symbols(['\\tau_{2%s}'%s for s in sides])
tau3 = sym.symbols(['\\tau_{3%s}'%s for s in sides])
GRFx = sym.symbols([    'G_{x%s}'%s for s in sides])
GRFy = sym.symbols([    'G_{y%s}'%s for s in sides])


tau1_l1, tau1_l2 = [],[]
tau2_l2, tau2_l3 = [],[]
tau3_l3, tau3_l4 = [],[]
GRF_r, GRFth, GRF_l4 = [],[],[]
Qtau1, Qtau2, Qtau3, QGRFs = [],[],[],[]
Qtot = sym.Matrix([[0],[0],[0],[0],[0],[0],[0],[0],[0]])

for i in range(len(sides)):
    tau1_l1.append(sym.Matrix([[0],[0],[-tau1[i]]]))
    tau1_l2.append(sym.Matrix([[0],[0], [tau1[i]]]))

    tau2_l2.append(sym.Matrix([[0],[0],[-tau2[i]]]))
    tau2_l3.append(sym.Matrix([[0],[0], [tau2[i]]]))

    tau3_l3.append(sym.Matrix([[0],[0],[-tau3[i]]]))
    tau3_l4.append(sym.Matrix([[0],[0], [tau3[i]]]))

    GRF_r.append(WeirdRot((1-d4)*rfoot_4, th1+th2[i]+th3[i]+th4[i]))
    GRFth.append(GRF_r[i][0]*GRFy[i] - GRF_r[i][1]*GRFx[i])
    GRF_l4.append(sym.Matrix([[GRFx[i]],[GRFy[i]],[GRFth[i]]]))

    Qtau1.append(     r1_0.jacobian(q).T*tau1_l1[i] + r2_0[i].jacobian(q).T*tau1_l2[i])
    Qtau2.append(  r2_0[i].jacobian(q).T*tau2_l2[i] + r3_0[i].jacobian(q).T*tau2_l3[i])
    Qtau3.append(  r3_0[i].jacobian(q).T*tau3_l3[i] + r4_0[i].jacobian(q).T*tau3_l4[i])
    QGRFs.append(rfoot_0[i].jacobian(q).T*GRF_l4[i])

    Qtot = Qtot + Qtau1[i] + Qtau2[i] + Qtau3[i] + QGRFs[i]

Qtot = mp_trigsimp(Qtot)
process_timer = log_time(process_timer, "Simplified Qtot")
# ------------------------------------------------------------------------------------------------------------------
# Mass, Gravity, Coriolis matrices, EOM
LgM = sym.hessian(T, dq)
LgM = mp_trigsimp(LgM)
process_timer = log_time(process_timer, "Simplified LgM")

LgG = V.jacobian(q)

process_timer = log_time()
dM = LgM.reshape(LgM.rows * LgM.cols, 1)
dM = dM.jacobian(q)*dq
dM = dM.reshape(LgM.rows, LgM.cols)
dM = mp_trigsimp(dM)
process_timer = log_time(process_timer, "Simplified dM")
Tj = T.jacobian(q)
Tj = mp_trigsimp(Tj)
process_timer = log_time(process_timer, "Simplified Tj")
LgC = dM*dq - Tj.T

EOM = LgM*ddq + LgC + LgG.T - Qtot

Timer initialized
Simplified 'r's: 6.916456 sec
Simplified T: 69.800657 sec
Simplified Qtot: 11.877412 sec
Simplified LgM: 28.045310 sec
Timer initialized
Simplified dM: 78.427100 sec


In [ ]:
# unsimplified, simplified unedited
# \displaystyle \left[\begin{matrix}- G_{xL} - G_{xR} + \ddot{\theta}_{1} \left(- 1.0 d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{3L} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \ddot{\theta}_{3R} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{x} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(\dot{\theta}_{1} \left(1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\\- G_{yL} - G_{yR} + \ddot{\theta}_{1} \left(- 1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{3L} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{3R} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{y} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(1.0 \dot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{2L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{2R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + g m_{1} + 2 g m_{2} + 2 g m_{3} + 2 g m_{4}\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{xR} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{1} + 2.0 I_{2} + 2.0 I_{3} + 2.0 I_{4} + 1.0 d_{1}^{2} l_{1}^{2} m_{1} + 2.0 d_{2}^{2} l_{2}^{2} m_{2} + 2.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 2.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 2.0 l_{2}^{2} m_{3} + 2.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 2.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2L} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2R} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{x} \left(- 1.0 d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{y} \left(- 1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{1} \dot{x} d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{2L} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{2R} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} \left(- 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) + \dot{x} \left(\dot{\theta}_{1} \left(1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{y} \left(1.0 \dot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - d_{1} g l_{1} m_{1} \sin{\left(\theta_{1} \right)} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2L} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{x} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{y} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{1} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{2L} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{y} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - \tau_{1L} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{xR} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2R} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{x} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{y} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} \left(- 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{2R} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} \left(- 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{y} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - \tau_{1R} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} \left(l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} \left(l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{x} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \ddot{y} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{1} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{y} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{2L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - \tau_{2L} + d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{1} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{2L} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{3L} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{y}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{2L} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{3L} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{x}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - m_{4} \left(\dot{\theta}_{1} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 \dot{y}\right) - 0.5 m_{4} \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{y}\right) \left(- \dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - 0.5 m_{4} \left(2 \dot{\theta}_{1} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 2 \dot{\theta}_{1} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2 \dot{\theta}_{2L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 2 \dot{\theta}_{2L} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2 \dot{\theta}_{3L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 2 \dot{\theta}_{3L} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{x}\right)\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{xR} \left(l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} \left(l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{x} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{y} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{1} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{y} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{2R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - \tau_{2R} + d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{2R} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{3R} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{y}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{2R} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{3R} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{x}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - m_{4} \left(\dot{\theta}_{1} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 \dot{y}\right) - 0.5 m_{4} \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{y}\right) \left(- \dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 0.5 m_{4} \left(2 \dot{\theta}_{1} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 2 \dot{\theta}_{1} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2 \dot{\theta}_{2R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 2 \dot{\theta}_{2R} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2 \dot{\theta}_{3R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 2 \dot{\theta}_{3R} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{x}\right)\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \ddot{\theta}_{1} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{2L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{3L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{2L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \tau_{3L} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{\theta}_{1} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{2R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{3R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{2R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \tau_{3R} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\end{matrix}\right]
# \displaystyle \left[\begin{matrix}- G_{xL} - G_{xR} + 1.0 \ddot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \ddot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \ddot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{x} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(\dot{\theta}_{1} \left(1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\\- G_{yL} - G_{yR} + \ddot{\theta}_{1} \left(- 1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{3L} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{3R} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{y} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(1.0 \dot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{2L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{2R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + g m_{1} + 2 g m_{2} + 2 g m_{3} + 2 g m_{4}\\- 1.0 G_{xL} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xL} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{xL} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2.0 G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xR} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{xR} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{xR} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2.0 G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 G_{yL} d_{4} l_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} - 1.0 G_{yL} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{yL} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} + \theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 2.0 G_{yL} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 G_{yR} d_{4} l_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} - 1.0 G_{yR} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{yR} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} + \theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} - 2.0 G_{yR} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 1.0 I_{1} \ddot{\theta}_{1} + 2.0 I_{2} \ddot{\theta}_{1} + 1.0 I_{2} \ddot{\theta}_{2L} + 1.0 I_{2} \ddot{\theta}_{2R} + 2.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{3L} + 1.0 I_{3} \ddot{\theta}_{3R} + 2.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2L} + 1.0 I_{4} \ddot{\theta}_{2R} + 1.0 I_{4} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{4L} + 1.0 I_{4} \ddot{\theta}_{4R} + 1.0 \ddot{\theta}_{1} d_{1}^{2} l_{1}^{2} m_{1} + 2.0 \ddot{\theta}_{1} d_{2}^{2} l_{2}^{2} m_{2} + 2.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 2.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 2.0 \ddot{\theta}_{1} l_{2}^{2} m_{3} + 2.0 \ddot{\theta}_{1} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 2.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2L} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2L} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{2L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2R} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2R} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{2R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{3L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{3L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{3R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{3R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{4R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} - 1.0 \ddot{x} d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 d_{1} g l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\\- 1.0 G_{xL} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xL} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{xL} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2.0 G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 G_{yL} d_{4} l_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} - 1.0 G_{yL} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{yL} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} + \theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 2.0 G_{yL} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 1.0 I_{2} \ddot{\theta}_{1} + 1.0 I_{2} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2L} + 1.0 I_{4} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{4L} + 1.0 \ddot{\theta}_{1} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2L} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2L} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{2L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{3L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{3L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \tau_{1L} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\\- 1.0 G_{xR} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{xR} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{xR} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2.0 G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 G_{yR} d_{4} l_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} - 1.0 G_{yR} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{yR} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} + \theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} - 2.0 G_{yR} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 1.0 I_{2} \ddot{\theta}_{1} + 1.0 I_{2} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2R} + 1.0 I_{4} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{4R} + 1.0 \ddot{\theta}_{1} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2R} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2R} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{2R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{3R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{3R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \tau_{1R} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\\- 1.0 G_{xL} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xL} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2.0 G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 G_{yL} d_{4} l_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} - 1.0 G_{yL} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} + \theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 2.0 G_{yL} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2L} + 1.0 I_{4} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{4L} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{2L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{2L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3L} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{3L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{3L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 1.0 \dot{\theta}_{1}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} + 1.0 \dot{\theta}_{2L}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{2L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 1.0 \dot{\theta}_{2L}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \tau_{2L} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\\- 1.0 G_{xR} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{xR} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2.0 G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 G_{yR} d_{4} l_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} - 1.0 G_{yR} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} + \theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} - 2.0 G_{yR} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2R} + 1.0 I_{4} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{4R} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{2R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{2R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3R} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{3R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{3R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 1.0 \dot{\theta}_{1}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} + 1.0 \dot{\theta}_{2R}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{2R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 1.0 \dot{\theta}_{2R}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \tau_{2R} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{\theta}_{1} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + 1.0 \ddot{\theta}_{2L} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + 1.0 \ddot{\theta}_{3L} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{2L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \tau_{3L} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{\theta}_{1} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + 1.0 \ddot{\theta}_{2R} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + 1.0 \ddot{\theta}_{3R} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{2R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \tau_{3R} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\end{matrix}\right]


# edited
# \displaystyle \left[\begin{matrix}- G_{xL} - G_{xR} + \ddot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{3L} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \ddot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{x} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(\dot{\theta}_{1} \left(1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\\- G_{yL} - G_{yR} + \ddot{\theta}_{1} \left(- 1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{3L} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{3R} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{y} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(1.0 \dot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{2L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{2R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + g m_{1} + 2 g m_{2} + 2 g m_{3} + 2 g m_{4}\\ G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} +        l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{xR} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{1} + 2.0 I_{2} + 2.0 I_{3} + 2.0 I_{4} + 1.0 d_{1}^{2} l_{1}^{2} m_{1} + 2.0 d_{2}^{2} l_{2}^{2} m_{2} + 2.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 2.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 2.0 l_{2}^{2} m_{3} + 2.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 2.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2L} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2R} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{x} \left(- 1.0 d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{y} \left(- 1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{1} \dot{x} d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{2L} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{2R} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} \left(- 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) + \dot{x} \left(\dot{\theta}_{1} \left(1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{y} \left(1.0 \dot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - d_{1} g l_{1} m_{1} \sin{\left(\theta_{1} \right)} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2L} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{x} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{y} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{1} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{2L} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \dot{\theta}_{2L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{2L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 \dot{\theta}_{2L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} \left(- 2.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - 2.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{3L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{4L} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{y} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - \tau_{1L} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{xR} \left(l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} \left(l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2R} \left(1.0 I_{2} + 1.0 I_{3} + 1.0 I_{4} + 1.0 d_{2}^{2} l_{2}^{2} m_{2} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 2.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2}^{2} m_{3} + 1.0 l_{2}^{2} m_{4} + 2.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{x} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{y} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{1} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{1} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{1} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} \left(- 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{2R} \dot{x} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \dot{\theta}_{2R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{2R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 \dot{\theta}_{2R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} \left(- 2.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - 2.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \dot{x} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{x} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{3R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R} \dot{y} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{x} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{4R} \dot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{y} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - \tau_{1R} + d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + g m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} \left(l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} \left(l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3L} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{x} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \ddot{y} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{1} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 \dot{\theta}_{3L} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \sin{\left(\theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 l_{3} \cos{\left(\theta_{4L} \right)}\right)\right) - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{y} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{2L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - \tau_{2L} + d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{1} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{2L} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{3L} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{y}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{2L} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + \dot{\theta}_{3L} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{x}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - m_{4} \left(\dot{\theta}_{1} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 \dot{y}\right) - 0.5 m_{4} \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{y}\right) \left(- \dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) - 0.5 m_{4} \left(2 \dot{\theta}_{1} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 2 \dot{\theta}_{1} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2 \dot{\theta}_{2L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 2 \dot{\theta}_{2L} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2 \dot{\theta}_{3L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 2 \dot{\theta}_{3L} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{x}\right)\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{xR} \left(l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} \left(l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \ddot{\theta}_{1} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{2R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{3R} \left(1.0 I_{3} + 1.0 I_{4} + 1.0 d_{3}^{2} l_{3}^{2} m_{3} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 2.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 l_{3}^{2} m_{4}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{x} \left(1.0 d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{y} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{1} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 \dot{\theta}_{3R} l_{2} \left(d_{3} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + d_{4} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \sin{\left(\theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(1.0 l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 l_{3} \cos{\left(\theta_{4R} \right)}\right)\right) - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} + \dot{x} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{y} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{2R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - \tau_{2R} + d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{2R} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{3R} d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{y}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 d_{3} l_{3} m_{3} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R}\right) \left(\dot{\theta}_{1} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{2R} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + \dot{\theta}_{3R} d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{x}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + g m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - m_{4} \left(\dot{\theta}_{1} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 \dot{y}\right) - 0.5 m_{4} \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{y}\right) \left(- \dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 0.5 m_{4} \left(2 \dot{\theta}_{1} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 2 \dot{\theta}_{1} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2 \dot{\theta}_{2R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 2 \dot{\theta}_{2R} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2 \dot{\theta}_{3R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 2 \dot{\theta}_{3R} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{x}\right)\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \ddot{\theta}_{1} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{2L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{3L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{2L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \tau_{3L} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{\theta}_{1} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{2R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{3R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4} - 1.0 d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{2R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \tau_{3R} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\end{matrix}\right]
# \displaystyle \left[\begin{matrix}- G_{xL} - G_{xR} + \ddot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \ddot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \ddot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{x} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(\dot{\theta}_{1} \left(1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(- 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 m_{4} \left(- d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(\dot{\theta}_{1} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(- 1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\\- G_{yL} - G_{yR} + \ddot{\theta}_{1} \left(- 1.0 d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)}\right) + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \ddot{\theta}_{3L} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \ddot{\theta}_{3R} \left(1.0 d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + m_{4} \left(0.5 d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 m_{4} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \ddot{\theta}_{4L} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{\theta}_{4R} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \ddot{y} \left(1.0 m_{1} + 2.0 m_{2} + 2.0 m_{3} + 2.0 m_{4}\right) + \dot{\theta}_{1} \left(1.0 \dot{\theta}_{1} \left(- d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{2L} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + \dot{\theta}_{2L} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right)\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{2R} \left(\dot{\theta}_{1} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + \dot{\theta}_{2R} \left(1.0 d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 m_{3} \left(d_{3} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)}\right) + 1.0 m_{4} \left(- d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right)\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) + \dot{\theta}_{3L} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{2L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 1.0 \dot{\theta}_{3L} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\right) + \dot{\theta}_{3R} \left(1.0 \dot{\theta}_{1} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{2R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 1.0 \dot{\theta}_{3R} \left(d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{4L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + g m_{1} + 2 g m_{2} + 2 g m_{3} + 2 g m_{4}\\-G_{xL} d_{4} l_{4} \sin{\left(\theta_{1}                  + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{xL} l_{2}       \cos{\left(\theta_{1} + \theta_{2L} \right)} - G_{xL} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2.0 G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xR} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{xR} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{xR} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2.0 G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 G_{yL} d_{4} l_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} - 1.0 G_{yL} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{yL} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} + \theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 2.0 G_{yL} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 G_{yR} d_{4} l_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} - 1.0 G_{yR} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{yR} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} + \theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} - 2.0 G_{yR} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 1.0 I_{1} \ddot{\theta}_{1} + 2.0 I_{2} \ddot{\theta}_{1} + 1.0 I_{2} \ddot{\theta}_{2L} + 1.0 I_{2} \ddot{\theta}_{2R} + 2.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{3L} + 1.0 I_{3} \ddot{\theta}_{3R} + 2.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2L} + 1.0 I_{4} \ddot{\theta}_{2R} + 1.0 I_{4} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{4L} + 1.0 I_{4} \ddot{\theta}_{4R} + 1.0 \ddot{\theta}_{1} d_{1}^{2} l_{1}^{2} m_{1} + 2.0 \ddot{\theta}_{1} d_{2}^{2} l_{2}^{2} m_{2} + 2.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 2.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 2.0 \ddot{\theta}_{1} l_{2}^{2} m_{3} + 2.0 \ddot{\theta}_{1} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 2.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2L} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2L} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{2L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2R} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2R} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{2R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{3L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{3L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{3R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{3R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{4R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} - 1.0 \ddot{x} d_{1} l_{1} m_{1} \cos{\left(\theta_{1} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{1} l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 d_{1} g l_{1} m_{1} \sin{\left(\theta_{1} \right)} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\\- 1.0 G_{xL} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xL} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{xL} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2.0 G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 G_{yL} d_{4} l_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} - 1.0 G_{yL} l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} - 1.0 G_{yL} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} + \theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 2.0 G_{yL} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 1.0 I_{2} \ddot{\theta}_{1} + 1.0 I_{2} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2L} + 1.0 I_{4} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{4L} + 1.0 \ddot{\theta}_{1} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2L} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2L} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{2L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2L} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{3L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{3L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{3L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{3L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} - 1.0 \dot{\theta}_{3L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{3L}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \tau_{1L} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2L} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\\- 1.0 G_{xR} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{xR} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{xR} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2.0 G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 G_{yR} d_{4} l_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} - 1.0 G_{yR} l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} - 1.0 G_{yR} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} + \theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} - 2.0 G_{yR} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 1.0 I_{2} \ddot{\theta}_{1} + 1.0 I_{2} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2R} + 1.0 I_{4} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{4R} + 1.0 \ddot{\theta}_{1} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{1} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2R} d_{2}^{2} l_{2}^{2} m_{2} + 1.0 \ddot{\theta}_{2R} d_{3}^{2} l_{3}^{2} m_{3} + 2.0 \ddot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{2R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{3} + 1.0 \ddot{\theta}_{2R} l_{2}^{2} m_{4} + 2.0 \ddot{\theta}_{2R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{3R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{3R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{3R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{x} d_{2} l_{2} m_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{x} l_{2} m_{3} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{2} m_{4} \cos{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \ddot{y} d_{2} l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \ddot{y} l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{3R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} - 1.0 \dot{\theta}_{3R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{3R}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \tau_{1R} + 1.0 d_{2} g l_{2} m_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 g l_{2} m_{3} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{2} m_{4} \sin{\left(\theta_{1} + \theta_{2R} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\\- 1.0 G_{xL} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{xL} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 2.0 G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yL} d_{4} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 G_{yL} d_{4} l_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} - 1.0 G_{yL} l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} + \theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yL} l_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} - 2.0 G_{yL} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} \cos{\left(\theta_{4L} \right)} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2L} + 1.0 I_{3} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2L} + 1.0 I_{4} \ddot{\theta}_{3L} + 1.0 I_{4} \ddot{\theta}_{4L} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - 2.0 \ddot{\theta}_{2L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{2L} l_{2} l_{3} m_{4} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{\theta}_{2L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3L} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3L} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{3L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{\theta}_{3L} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4L} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 1.0 \dot{\theta}_{1}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2L} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2L} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2L} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} + 1.0 \dot{\theta}_{2L}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3L} \right)} + 1.0 \dot{\theta}_{2L}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + 1.0 \dot{\theta}_{2L}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3L} \right)} - 2.0 \dot{\theta}_{2L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 2.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{\theta}_{4L}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \tau_{2L} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2L} \right)} \cos{\left(\theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3L} \right)} \sin{\left(\theta_{2L} + \theta_{4L} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4L} \right)} \sin{\left(\theta_{1} + \theta_{2L} \right)} \cos{\left(\theta_{3L} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2L} \right)} \cos{\left(\theta_{4L} \right)} \cos{\left(\theta_{1} + \theta_{3L} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\\- 1.0 G_{xR} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{xR} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 2.0 G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 G_{yR} d_{4} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 G_{yR} d_{4} l_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} - 1.0 G_{yR} l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} + \theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} + 2.0 G_{yR} l_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} - 2.0 G_{yR} l_{4} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} \cos{\left(\theta_{4R} \right)} + 1.0 I_{3} \ddot{\theta}_{1} + 1.0 I_{3} \ddot{\theta}_{2R} + 1.0 I_{3} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{1} + 1.0 I_{4} \ddot{\theta}_{2R} + 1.0 I_{4} \ddot{\theta}_{3R} + 1.0 I_{4} \ddot{\theta}_{4R} + 1.0 \ddot{\theta}_{1} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{1} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{1} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{1} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{1} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{1} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{2R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - 2.0 \ddot{\theta}_{2R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{2R} l_{2} l_{3} m_{4} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{\theta}_{2R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{3R} d_{3}^{2} l_{3}^{2} m_{3} + 1.0 \ddot{\theta}_{3R} d_{4}^{2} l_{4}^{2} m_{4} - 2.0 \ddot{\theta}_{3R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{\theta}_{3R} l_{3}^{2} m_{4} + 1.0 \ddot{\theta}_{4R} d_{4}^{2} l_{4}^{2} m_{4} - 1.0 \ddot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} + 1.0 \ddot{x} d_{3} l_{3} m_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{x} l_{3} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \ddot{y} d_{3} l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 \ddot{y} d_{4} l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 \ddot{y} l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{1}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 1.0 \dot{\theta}_{1}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2R} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2R} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 2.0 \dot{\theta}_{1} \dot{\theta}_{2R} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{1} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} + 1.0 \dot{\theta}_{2R}^{2} d_{3} l_{2} l_{3} m_{3} \sin{\left(\theta_{3R} \right)} + 1.0 \dot{\theta}_{2R}^{2} d_{4} l_{2} l_{4} m_{4} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + 1.0 \dot{\theta}_{2R}^{2} l_{2} l_{3} m_{4} \sin{\left(\theta_{3R} \right)} - 2.0 \dot{\theta}_{2R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 2.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{\theta}_{4R}^{2} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \tau_{2R} + 1.0 d_{3} g l_{3} m_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2R} \right)} \cos{\left(\theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{3R} \right)} \sin{\left(\theta_{2R} + \theta_{4R} \right)} \cos{\left(\theta_{1} \right)} - 1.0 d_{4} g l_{4} m_{4} \sin{\left(\theta_{4R} \right)} \sin{\left(\theta_{1} + \theta_{2R} \right)} \cos{\left(\theta_{3R} \right)} + 1.0 d_{4} g l_{4} m_{4} \cos{\left(\theta_{2R} \right)} \cos{\left(\theta_{4R} \right)} \cos{\left(\theta_{1} + \theta_{3R} \right)} + 1.0 g l_{3} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\\G_{xL} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + G_{xL} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - G_{yL} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{\theta}_{1} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + 1.0 \ddot{\theta}_{2L} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3L} + \theta_{4L} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + 1.0 \ddot{\theta}_{3L} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4L} \right)}\right) + \ddot{\theta}_{4L} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{2L} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{2} \cos{\left(\theta_{3L} + \theta_{4L} \right)} + \dot{\theta}_{4L} l_{3} \cos{\left(\theta_{4L} \right)}\right) - 1.0 \dot{\theta}_{3L} \dot{\theta}_{4L} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4L} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \tau_{3L} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + 0.5 \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{2L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{3L} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)}\right) + \dot{\theta}_{4L} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2L} + \dot{\theta}_{3L} + \dot{\theta}_{4L}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{2L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{2L} l_{2} \cos{\left(\theta_{1} + \theta_{2L} \right)} - \dot{\theta}_{2L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{3L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{\theta}_{3L} l_{3} \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} \right)} + \dot{\theta}_{4L} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2L} + \theta_{3L} + \theta_{4L} \right)}\\G_{xR} l_{4} \left(1 - d_{4}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + G_{xR} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \left(1 - d_{4}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - G_{yR} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{\theta}_{1} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + 1.0 \ddot{\theta}_{2R} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{2} l_{4} m_{4} \sin{\left(\theta_{3R} + \theta_{4R} \right)} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + 1.0 \ddot{\theta}_{3R} \left(I_{4} + d_{4}^{2} l_{4}^{2} m_{4} - d_{4} l_{3} l_{4} m_{4} \sin{\left(\theta_{4R} \right)}\right) + \ddot{\theta}_{4R} \left(1.0 I_{4} + 1.0 d_{4}^{2} l_{4}^{2} m_{4}\right) - 1.0 \ddot{x} d_{4} l_{4} m_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 1.0 \ddot{y} d_{4} l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{\theta}_{1} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{2R} d_{4} l_{4} m_{4} \left(\dot{\theta}_{3R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{2} \cos{\left(\theta_{3R} + \theta_{4R} \right)} + \dot{\theta}_{4R} l_{3} \cos{\left(\theta_{4R} \right)}\right) - 1.0 \dot{\theta}_{3R} \dot{\theta}_{4R} d_{4} l_{3} l_{4} m_{4} \cos{\left(\theta_{4R} \right)} - 1.0 \dot{x} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 \dot{y} d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \tau_{3R} + d_{4} g l_{4} m_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(0.5 \dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + 0.5 \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + 0.5 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{2R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{2} \sin{\left(\theta_{1} + \theta_{2R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{3R} \left(d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + l_{3} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)}\right) + \dot{\theta}_{4R} d_{4} l_{4} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} + \dot{y}\right) \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - 1.0 d_{4} l_{4} m_{4} \left(\dot{\theta}_{1} + \dot{\theta}_{2R} + \dot{\theta}_{3R} + \dot{\theta}_{4R}\right) \left(\dot{\theta}_{1} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{1} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{1} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{2R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{2R} l_{2} \cos{\left(\theta_{1} + \theta_{2R} \right)} - \dot{\theta}_{2R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{3R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{\theta}_{3R} l_{3} \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} \right)} + \dot{\theta}_{4R} d_{4} l_{4} \sin{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)} - \dot{x}\right) \cos{\left(\theta_{1} + \theta_{2R} + \theta_{3R} + \theta_{4R} \right)}\end{matrix}\right]

In [ ]:
# # LAMBDIFY ------------------------------------------------------------------------------------------------------------------    
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos}


sym_list = [g,
            m1 , m2, m3, m4,
            l1 , l2, l3, l4,
            In1,In2,In3,In4,
            d1 , d2, d3, d4,
            x  ,   y,  th1] +   th2 +   th3 +   th4 +\
            [dx,  dy, dth1] +  dth2 +  dth3 +  dth4 +\
            [ddx,ddy,ddth1] + ddth2 + ddth3 + ddth4 +\
            tau1 + tau2 + tau3 +\
            GRFx + GRFy



# DOFs
DOF0 = [('x','b'),('y','b'),('th1','b')]
DOF1 = ['th2','th3','th4']

DOFs  = [(  'x','b'),
         (  'y','b'),
         ('th1','b'),
         ('th2','L'),
         ('th2','R'),
         ('th3','L'),
         ('th3','R'),
         ('th4','L'),
         ('th4','R')]
TDOFs = ['x','y']

links = ['b','l1','l2','l3']
joints = ['hip','knee','ankle']

lamb_EOM  = {}
for dof_i, dof in enumerate(DOFs):
    lamb_EOM[dof]    = sym.lambdify(sym_list, EOM[dof_i],modules = [func_map])
    
lamb_footp = {}
lamb_footv = {}

lamb_massp = {}
lamb_jointp = {}

for s_i, s in enumerate(sides):
    for dof_i, dof in enumerate(TDOFs):
        lamb_footp[dof,s]  = sym.lambdify(sym_list,rfoot_0[s_i][dof_i],modules = [func_map])
        lamb_footv[dof,s]  = sym.lambdify(sym_list, drfoot[s_i][dof_i],modules = [func_map])
        
        lamb_jointy.update({('shoulder',side): sym.lambdify(sym_list,p_shoulder[1],modules =  [func_map])})
        
        lamb_massp.update({('b',dof,s)  : sym.lambdify(sym_list,r1_0[s_i][dof_i],modules = [func_map])})
        lamb_massp.update({('l1',dof,s) : sym.lambdify(sym_list,r2_0[s_i][dof_i],modules = [func_map])})
        lamb_massp.update({('l2',dof,s) : sym.lambdify(sym_list,r3_0[s_i][dof_i],modules = [func_map])})
        lamb_massp.update({('l3',dof,s) : sym.lambdify(sym_list,r4_0[s_i][dof_i],modules = [func_map])})
        
        lamb_jointp.update({('hip',dof,s)   : sym.lambdify(sym_list,  rhip[s_i][dof_i],modules = [func_map])})
        lamb_jointp.update({('knee',dof,s)  : sym.lambdify(sym_list, rknee[s_i][dof_i],modules = [func_map])})
        lamb_jointp.update({('ankle',dof,s) : sym.lambdify(sym_list,rankle[s_i][dof_i],modules = [func_map])})

In [ ]:
# PARAMETERS:
mass = {'b': 0.5,
       'l1': 0.2,
       'l2': 0.2,
       'l3': 0.1}

length = {'b': 1,
         'l1': 0.5,
         'l2': 0.5,
         'l3': 0.5}

dCOM = {'b': 0.5,
       'l1': 0.4,
       'l2': 0.4,
       'l3': 0.2}

inertia = {'b': mass['b']*length['b']**2/12,
          'l1': mass['l1']*length['l1']**2/12,
          'l2': mass['l2']*length['l2']**2/12,
          'l3': mass['l3']*length['l3']**2/12,}

def get_var_list(m,n):
    var_list = [m.g]+\
               [m.m[l] for l in links]+\
               [m.len[l] for l in links]+\
               [m.In[l] for l in links]+\
               [m.dCOM[l] for l in links]+\
               [m.q[n,dof] for dof in DOFs]+\
               [m.dq[n,dof] for dof in DOFs]+\
               [m.ddq[n,dof] for dof in DOFs]+\
               [m.tau[n,j,s] for s in sides for j in joints]+\
               [m.GRF[n,dof,s] for s in sides for dof in TDOFs]

    return var_list


In [ ]:
def create_model(N,hm,mu):
    m = ConcreteModel()
    
    #SETS---------------------------------
    m.N =RangeSet(N)
    m.L =Set(initialize = links)
    m.J = Set(initialize = joints)
    m.DOF = Set(initialize= DOFs)
    m.TDOF = Set(initialize = TDOFs)
    m.S = Set(initialize = sides)
    
    #PARAMS-------------------------------
    m.g = Param(initialize = 9.81)
    m.mu = Param(initialize = mu)
    
    m.m = Param(m.L, initialize = mass)
    m.len = Param(m.L, initialize = length)
    m.In = Param(m.L, initialize = inertia)
    m.dCOM = Param(m.L, initialize = dCOM)
    
    
    signs = ['ps','ng']
    m.sgn = Set(initialize = signs)
    m.ground_constraints = ['contact','sliding_ps','sliding_ng']
    
    #VARIABLES----------------------------
    #gen coordinates
    m.q = Var(m.N,m.DOF)
    m.dq = Var(m.N,m.DOF)
    m.ddq = Var(m.N,m.DOF)
    
    #forces
    m.tau = Var(m.N,m.J   ,m.S)
    m.GRF = Var(m.N,m.TDOF,m.S)

# ------------------------------------------------------------------------------------------------------------------
    #TIME AND INTEGRATION

    # timestep
    m.hm = Param(initialize = hm, mutable = True) # master timestep
    m.h = Var(m.N, bounds = (0.8,1.0))
    
    # Integration constraints 
    def BwEuler_p(m,n,dof1,dof2): # for positions
        dof = (dof1,dof2)
        if n > 1:
            return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
        else:
            return Constraint.Skip 
    m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

    def BwEuler_v(m,n,dof1,dof2): # for velocities
        dof = (dof1,dof2)
        if n > 1:
            return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
        else:
            return Constraint.Skip 
    m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)
    
    def dynamics(m,n,dof1,dof2):
        dof = (dof1,dof2)
        var_list = get_var_list(m,n)
        temp = lamb_EOM[dof](*var_list) == 0
#         if temp in [None, True, False]:
#             return Constraint.Skip
#         else:
        return temp
    m.dynamics = Constraint(m.N,m.DOF, rule = dynamics)


# ------------------------------------------------------------------------------------------------------------------
    # Link Bounds ------------------------
    for n in range(1,N+1):
#         m.tau[n,'hip'].setlb(-20)
#         m.tau[n,'hip'].setub(20)
#         m.tau[n,'knee'].setlb(-10)
#         m.tau[n,'knee'].setub(10)
#         m.tau[n,'ankle'].setlb(-10)
#         m.tau[n,'ankle'].setub(10)
        m.q[n,  'x','b'].setlb(0.0)
        m.q[n,  'y','b'].setlb(0.0)
        m.q[n,'th1','b'].setlb(-np.pi/2)
        m.q[n,'th1','b'].setub(np.pi/2)
        for s in sides:
            # https://www.researchgate.net/figure/The-range-of-movements-in-the-hip-joint-a-flexion-extension-10-0-130_fig4_326636749
            m.q[n,'th2',s].setlb(-np.pi/2/9)
            m.q[n,'th2',s].setub(np.pi*3/4)
            #https://x10therapy.com/knee-range-of-motion/
            m.q[n,'th3',s].setlb(-np.pi*3/4)
            m.q[n,'th3',s].setub(0)
            #https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4994968/
            m.q[n,'th4',s].setlb(np.pi/6)
            m.q[n,'th4',s].setub(np.pi*7/12)
            # ranges of motion above are temporary!!!
            m.GRF[n,'y',s].setlb(0)
        

    # Joint and Mass Pos -----------------
    m.jointp = Var(m.N, m.J, m.TDOF, m.S, bounds = (0.0,None))
    def def_jointp(m,n,j,dof,s):
        var_list = get_var_list(m,n)
        return m.jointp[n,j,dof,s] == lamb_jointp[(j,dof,s)](*var_list)
    m.def_jointp = Constraint(m.N, m.J, m.TDOF, m.S, rule = def_jointp)
 
    m.massp = Var(m.N, m.L, m.TDOF, m.S, bounds = (0.0,None))
    def def_jointp(m,n,l,dof,s):
        var_list = get_var_list(m,n)
        return m.massp[n,l,dof,s] == lamb_massp[(l,dof,s)](*var_list)
    m.def_massp = Constraint(m.N, m.L, m.TDOF, m.S, rule = def_massp)

    #Ground Constraints ------------------
    m.footp = Var(m.N, m.TDOF, m.S, bounds = (0.0,None))
    def def_footp(m,n,dof,s):
        var_list = get_var_list(m,n)
        return m.footp[n,dof,s] == lamb_footp[(dof,s)](*var_list)
    m.def_footp = Constraint(m.N, m.TDOF, m.S, rule = def_footp)
        
    m.footv = Var(m.N, m.TDOF, m.S, m.sgn, bounds = (0.0,None))
    def def_footv(m,n,dof,s,sgn):
        var_list = get_var_list(m,n)
        return m.footv[n,dof,s,'ps'] - m.footv[n,dof,s,'ng'] == lamb_footv[(dof,s)](*var_list)
    m.def_footv = Constraint(m.N, m.TDOF, m.S, m.sgn, rule = def_footv)
    





    m.alpha = Var(m.N, m.S, bounds = (0,1))
    def friction(m,n,s):
        if n == 1:
            return Constraint.Skip
        return m.GRF[n,'x',s] == (1 - 2*m.alpha[n,s])*m.mu*m.GRF[n,'y',s]
    m.friction = Constraint(m.N, m.S, rule = friction)
    
    m.ground_penalty = Var(m.N, m.S, m.ground_constraints, bounds = (0.0,None))
    def ground_complementarity(m,n,s,gc):
        if n == 1:
            return Constraint.Skip
        if gc == 'contact':
            A = m.GRF[n,'y',s]
            B = m.footp[n,'y',s] + m.footv[n,dof,s,'ps'] + m.footv[n,dof,s,'ng']
            
        if gc == 'sliding_ps':
            A = 1 - m.alpha[n,s]
            B = m.footv[n,dof,s,'ps']
            
        if gc == 'sliding_ng':
            A = m.alpha[n,s]
            B = m.footv[n,dof,s,'ng']
        
        return A*B <= m.ground_penalty[n,s,gc]
    m.ground_complementarity = Constraint(m.N, m.S, m.ground_constraints, rule = ground_complementarity)
 



    # OBJECTIVES -----------------------------------------------------------------------------------------------------------
    def MinPenalty(m):
        return sum([m.ground_penalty[n,s,gc] for n in range(1,N+1) for s in m.S for gc in m.ground_constraints])
    m.MinPenalty = Objective(rule = MinPenalty)
    
    return m


In [ ]:
def init_opt(max_iter, max_time): # initializes solver
    opt = SolverFactory('ipopt') # standard issue, garden variety ipopt
    opt.options["linear_solver"] = 'ma97'
    opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
    opt.options["max_iter"] = max_iter # maximum number of iterations
    opt.options["max_cpu_time"] = max_time # maximum cpu time in seconds
    opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
    opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    opt.options["OF_ma86_scaling"] = 'none'
    
    return opt


In [ ]:
# HIGH DROP -----------------------------------------------------------------------------------------------------------------

def high_drop(m):
    # initial condition
    for dof in DOFs:
        #m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0) # rest
        m.q[  1,dof].fix(0) # neutral posture
    m.q[1,'y','b'].unfix()
    for s in sides:
        m.q[1,'th4',s].unfix()
        m.q[1,('th4',s)].fix(np.pi/3)
        m.footp[1,'y',s].fix(10)
    
    # during
    for n in range(1,N+1):
        for s in sides:
            for j in joints:
                m.tau[n,j,s].fix(0)

    return m

In [ ]:
# LOW DROP ------------------------------------------------------------------------------------------------------------------

def low_drop(m):
    # initial condition
    for dof in DOFs:
        m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0) # rest
        m.q[  1,dof].fix(0) # neutral posture
    m.q[1,'y','b'].unfix()
    for s in sides:
        m.q[1,'th4',s].unfix()
        m.footp[1,'y',s].setlb(0.1) # start near the ground

    # during
    for n in range(1,N+1):
        for s in sides:
            for j in joints:
                m.tau[n,j,s].fix(0) # no forces
            if n > 15:
                m.footp[15,'y',s].fix(0) # force it to land
    return m

In [ ]:
# # # STANDING ------------------------------------------------------------------------------------------------------------------
def standing(m):
    # initial condition
    for dof in DOFs:
        m.dq[1,dof].fix(0) # rest
        m.q[ 1,dof].fix(0) # neutral posture
    m.q[1,'y','b'].unfix()
    for s in sides:
        m.q[1,'th4',s].unfix()

    # during
    for n in range(1,N+1):
        for s in sides:
            m.footp[n,'y',s].fix(0) # must stay grounded for the full time

    # final condition (hold)
    for n in range(N-20,N+1):
        for dof in DOFs:
            m.dq[n,dof].fix(0)
        m.q[n,'th1','b'].fix(0)
    
    return m

In [ ]:
# # HIGH DROP + NUDGE ---------------------------------------------------------------------------------------------------------

def bounce(m):
    for dof in DOFs:
        m.dq[1,dof].fix(0)
    m.q[1,'x','b'].fix(0)
    for s in sides:
        m.footp[1,'y',s].fix(0)
        
    # middle
    for s in sides:
        m.footp[15,'y',s].setlb(0.2)
        for n in range(1,N+1):
            m.footp[n,'x',s].setlb(-0.5)
            m.footp[n,'x',s].setub(0.5)
    
    # end
    for n in range(N-10,N+1):
        for dof in DOFs:
            m.dq[n,dof].fix(0)
        m.q[n,'th1','b'].fix(0)
        for s in sides:
            m.footp[n,'y',s].fix(0)
                
    return m

In [ ]:
N = 50
hm = 0.02
mu = 1.0
m = create_model(N,hm,mu)

# m = high_drop(m)
# m = low_drop(m)
# m = standing(m)
m = bounce(m)

opt = init_opt(30000,3000)
results = opt.solve(m,tee=True)


In [ ]:
# if the problem is infeasible, this is how you can see which constraints weren't satisfied
from pyomo.util.infeasible import log_infeasible_constraints
log_infeasible_constraints(m)

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

xmax = np.max([m.q[n,'x','b'].value for n in range(1,N+1)])
ymax = np.max([m.q[n,'y','b'].value for n in range(1,N+1)])

def plot_model(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-1,xmax+1]) # adjust limits to solution
    ax.set_ylim([0,ymax+0.5])
    
    #plot body
    draw_width = 0.05
    body_xtl = m.q[i,'x','b'].value - (m.len[('b')])*sin(m.q[i,'th1','b'].value) - draw_width*cos(m.q[i,'th1','b'].value)
    body_ytl = m.q[i,'y','b'].value + (m.len[('b')])*cos(m.q[i,'th1','b'].value) - draw_width*sin(m.q[i,'th1','b'].value)
    body_xtr = m.q[i,'x','b'].value - (m.len[('b')])*sin(m.q[i,'th1','b'].value) + draw_width*cos(m.q[i,'th1','b'].value)
    body_ytr = m.q[i,'y','b'].value + (m.len[('b')])*cos(m.q[i,'th1','b'].value) + draw_width*sin(m.q[i,'th1','b'].value)
    body_xbl = m.q[i,'x','b'].value - draw_width*cos(m.q[i,'th1','b'].value)
    body_ybl = m.q[i,'y','b'].value - draw_width*sin(m.q[i,'th1','b'].value)
    body_xbr = m.q[i,'x','b'].value + draw_width*cos(m.q[i,'th1','b'].value)
    body_ybr = m.q[i,'y','b'].value + draw_width*sin(m.q[i,'th1','b'].value)
    ax.plot([body_xbl,body_xtl,body_xtr,body_xbr,body_xbl],[body_ybl,body_ytl,body_ytr,body_ybr,body_ybl],color='xkcd:blue')
    
    for s in sides:
        #plot thighs
        leg1_xt = m.jointp[ 'hip','x',s].value
        leg1_yt = m.jointp[ 'hip','y',s].value
        leg1_xb = m.jointp['knee','x',s].value
        leg1_yb = m.jointp['knee','y',s].value
        ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:green')

        #plot calves
        leg2_xt = m.jointp[ 'knee','x',s].value
        leg2_yt = m.jointp[ 'knee','y',s].value
        leg2_xb = m.jointp['ankle','x',s].value
        leg2_yb = m.jointp['ankle','y',s].value
        ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:black')

        #plot feet
        leg3_xt = m.jointp['ankle','x',s].value
        leg3_yt = m.jointp['ankle','y',s].value
        leg3_xb =  m.footp[        'x',s].value
        leg3_yb =  m.footp[        'y',s].value
        ax.plot([leg3_xt,leg3_xb],[leg3_yt,leg3_yb],color='xkcd:red')
    
update = lambda i: plot_model(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook


In [ ]:
m.GRF.pprint()